In [1]:
import pandas as pd
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import numpy as np
import folium

from pysal.lib import weights  
import segregation as seg

C:\Users\MorrisKurz\AppData\Local\Temp\ipykernel_12352\468827472.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


TODO: Documentation about neighborhood polygons and grids with pictures

In [3]:
# Read zensus data in 100m x 100m grid
df1 = pd.read_csv('../data_example/Zensus_Bremen_Buildings.csv', delimiter=';', index_col=0)
df2 = pd.read_csv('../data_example/Zensus_Bremen_Families.csv', delimiter=';', index_col=0)
df3 = pd.read_csv('../data_example/Zensus_Bremen_Households.csv', delimiter=';', index_col=0)
df4 = pd.read_csv('../data_example/Zensus_Bremen_Population.csv', delimiter=';', index_col=0)
df5 = gpd.read_file('../data_example/Zensus_Bremen_Grid_100m.gpkg')

In [14]:
# Merge them into a single dataframe by the column Grid_Code
merged_df = pd.merge(df1, df2, on='Grid_Code')
merged_df = pd.merge(merged_df, df3, on='Grid_Code')
merged_df = pd.merge(merged_df, df4, on='Grid_Code')
merged_df = pd.merge(merged_df, df5, on='Grid_Code')
zensus_data = gpd.GeoDataFrame(merged_df)

In [5]:
# Extract the grid middle points to match them later to their respective neighborhoods
zensus_data.geometry = zensus_data.geometry.centroid

In [6]:
# Load the neighborhood polygons
neighborhood_polygons = gpd.read_file('../data_example/Land_Prices_Neighborhood_Bremen.gpkg')

In [15]:
# Join the grid points into the neighborhoods and group them
zensus_data_by_neighborhood = gpd.sjoin(zensus_data, neighborhood_polygons, how='left', op='within')
zensus_data_by_neighborhood_grouped = zensus_data_by_neighborhood.groupby('Neighborhood_FID')

In [32]:
# Example question: How is the distribution of German residents vs. foreign residents in rich vs. poor neighborhoods?
# Get amount of people in neighborhoods
germans_nbgh = zensus_data_by_neighborhood_grouped.sk_germany.sum()
foreign_nbgh = zensus_data_by_neighborhood_grouped.sk_abroad.sum()
# Normalize by amount of buildings in the neighborhood
amount_of_buildings_nbgh = zensus_data_by_neighborhood_grouped.

In [24]:
zensus_data_by_neighborhood_grouped.buildings_total_units.sum()

Neighborhood_FID
1.0     2295
2.0     2531
3.0     3796
4.0       14
5.0        0
        ... 
84.0    1894
85.0    5568
86.0    1244
87.0     893
88.0    5168
Name: buildings_total_units, Length: 87, dtype: int64

In [27]:
zensus_data_by_neighborhood_grouped.households_total_units.sum()

Neighborhood_FID
1.0     2156
2.0     2425
3.0     3674
4.0       14
5.0        0
        ... 
84.0    1772
85.0    5215
86.0    1162
87.0     864
88.0    5020
Name: households_total_units, Length: 87, dtype: int64

In [28]:
zensus_data_by_neighborhood_grouped.sk_abroad.sum()

Neighborhood_FID
1.0      837
2.0      724
3.0      630
4.0       20
5.0        0
        ... 
84.0     166
85.0    1192
86.0      83
87.0      56
88.0     463
Name: sk_abroad, Length: 87, dtype: int64

In [29]:
zensus_data_by_neighborhood_grouped.size()

Neighborhood_FID
1.0       84
2.0       80
3.0       55
4.0     1346
5.0      692
        ... 
84.0     197
85.0     425
86.0     470
87.0     489
88.0    1708
Length: 87, dtype: int64

In [42]:
zensus_data_by_neighborhood_grouped.filter(lambda x: (germans_nbgh + foreign_nbgh <= 5)[x.name]).

,Grid_Code,buildings_total_units,n_occupied_by_owner,n_ownership_with_current_household,n_owned_without_current_household,n_rented_for_residential_purposes,n_rented_with_current_household,n_rented_without_current_household,n_vacation_and_leisure,n_vacant,...,sz_mult_foreign_only,sz_unknown,City_Code,geometry,index_right,Neighborhood_FID,Land_Value,Area_Types,Area_Count,City_Name
9424,100mN33305E42368,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4236800.000 3330500.000, 4236800.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
9425,100mN33305E42369,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4236900.000 3330500.000, 4236900.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
9596,100mN33306E42367,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4236700.000 3330600.000, 4236700.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
9597,100mN33306E42368,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4236800.000 3330600.000, 4236800.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
9598,100mN33306E42369,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4236900.000 3330600.000, 4236900.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18634,100mN33352E42337,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4233700.000 3335200.000, 4233700.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
18640,100mN33352E42343,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4234300.000 3335200.000, 4234300.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
18641,100mN33352E42344,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4234400.000 3335200.000, 4234400.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
18642,100mN33352E42345,0,0,0,0,0,0,0,0,0,...,0,0,04011000,"POLYGON ((4234500.000 3335200.000, 4234500.000...",4.0,5.0,90.3,CommercialBuildingArea_Residential,3.0,Bremen
